In [1]:
import sys                                                                                                                                                                                                              
sys.path[0]='/home/jovyan/mnt/neuronunit'                                                                                                                                                                                                     
from neuronunit.tests import get_neab                                                                                                                                                                                                         
import neuronunit                                                                                                                                                                                                                             
print(neuronunit.tests.__file__)                                                                                                                                                                                                              
from neuronunit.tests import utilities as outils                                                                                                                                                                                              
                                                                                                                                                                                                                                              
from neuronunit.tests import model_parameters as modelp                                                                                                                                                                                       
import numpy as np                                                                                                                                                                                                                            
model = outils.model                
from sciunit import scores


Getting Rheobase cached data value for from AIBS dataset 354190013
attempting to recover from pickled file
/home/jovyan/mnt/neuronunit/neuronunit/tests/__init__.py
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml
Mechanisms already loaded from path: /home/jovyan/mnt/neuronunit/neuronunit/tests/NeuroML2.  Aborting.
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml


In [2]:

class Score:
    def __init__(self, score):
        self.score = score

class Test:
    def _optimize(self, model,modelp):
        '''
        The implementation of optimization, consisting of implementation details.
        Inputs a model, and model parameter ranges to expore
        Private method for programmer designer.
        Outputs the optimal model, its attributes and the low error it resulted in.
        '''
        from neuronunit.tests import nsga
        gap = nsga.GAparams(model)
        # The number of generations is 2
        gap.NGEN = 2
        # The population of genes is 4
        gap.MU = 4
        
        gap.BOUND_LOW = [ np.min(i) for i in modelp.model_params.values() ]
        gap.BOUND_UP = [ np.max(i) for i in modelp.model_params.values() ]

        vmpop, pop, invalid_ind, pf = nsga.main(gap.NGEN,gap.MU,model,modelp)
        some_tuples = (vmpop, pop, invalid_ind, pf)
        return pop[0], some_tuples
        
    def _get_optimization_parameters(self, some_tuples):
        vmpop = some_tuples[0]
        # Your specific unpacking of tuples that _optimize returns
        scores = [ i.score for i in vmpop ]
        pop = some_tuples[1]
        
        errors = pop[0].fitness
        parameters = vmpop[0].attrs
      
        #parameters,errors,scores,_ = zip(*some_tuples)
        return parameters,scores

    def optimize(self, model, modelp):
        '''
        The Class users version of optimize
        where details are hidden in _optimizae
        '''

        # Do optimization including repeated calls to judge
        models, some_tuples = self._optimize(model,modelp)
        parameters, scores = self._get_optimization_parameters(some_tuples)
        # Maybe rebuild the original model
        # (i.e. restore the true class from the virtual version)

        # Your code keeps parameter sets and associated scores
        # All the organizing stuff
        
        
        # this a way of looking at solved model parameters, ie candidate solutions from 
        # the pareto front.

        
        # Make a ScoreArray (which is basically a pandas dataframe)
       
        return model, scores, some_tuples

    
t = Test()


model,scores,some_tuples = t.optimize(model,modelp)   

<module 'neuronunit.tests.model_parameters' from '/home/jovyan/mnt/neuronunit/neuronunit/tests/model_parameters.py'>
0 39.3715553272
1 0.0518254196574
2 9.93194568776e-05
3 -67.3740489831
4 -59.5125484902
5 -4.47762006555e-09
6 0.671900527826
7 -55.6996968732
8 -36.813955781
9 0.00115780964351
0 31.2834489251
1 0.102866164199
2 0.000100369035346
3 -62.6048389111
4 -56.1836270346
5 -3.62736259776e-09
6 0.876292526286
7 -47.9021626852
8 -32.8189238319
9 0.001221273142
0 32.4416452854
1 0.165787750135
2 9.30449495022e-05
3 -65.3603349633
4 -57.2497391043
5 -4.75915425424e-09
6 0.705821414254
7 -69.0494324109
8 -48.456174567
9 0.00094115889357
0 30.6983513151
1 0.125797651601
2 0.000103328342567
3 -73.6782396495
4 -56.4972671307
5 -4.88912359869e-09
6 0.285583373448
7 -61.2128860714


/opt/conda/lib/python3.5/site-packages/scoop-0.7.2.0-py3.5.egg/scoop/fallbacks.py:46: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().


8 -32.2511342188
9 0.000836911868153
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
<class 'float'>
current 270.60185185185185 spikes 1
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
<class 'float'>
current 271.0648148148148 spikes 1
{}
<class 'float'>
current 271.18055555555554 spikes 1
{}
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
<class 'float'>
current 67.5925925925926 spikes 1
{}
{}
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
<class 'float'>
current 359.49074074074076 spikes 1
<class 'float'>
checkpoint 1 output from parallel map [<neuronunit.tests.utilities.VirtualModel object at 0x7f59f2af8390>, <neuronunit.tests.utilities.VirtualModel object at 0x7f59f2af8d30>, <neuronunit.tests.utilities.VirtualModel object at 0x7f59f29b8048>, <neuronunit.tests.utilities.VirtualModel obje

{}
<class 'sciunit.scores.ZScore'>


{}


...

{}


{}


{}


{}


0 [32.441645285436714, 0.16578775013484415, 9.3044949502172592e-05, -65.360334963327048, -57.249739104278476, -4.7591542542417675e-09, 0.70582141425425926, -69.049432410868235, -48.456174567028214, 0.00094115889356982695]
<class 'int'> <class 'deap.creator.Individual'>
1 [39.371555327244685, 0.05182541965743797, 9.9319456877608665e-05, -67.374048983091399, -59.512548490187804, -4.4776200655456835e-09, 0.67190052782571763, -55.69969687317969, -36.813955780999962, 0.0011578096435100793]
<class 'int'> <class 'deap.creator.Individual'>
2 [30.698351315120032, 0.12579765160058567, 0.00010332834256650572, -73.678239649537971, -56.49726713070536, -4.8891235986886736e-09, 0.28558337344837531, -61.212886071387821, -32.251134218815551, 0.00083691186815277392]
<class 'int'> <class 'deap.creator.Individual'>
3 [31.283448925127193, 0.10286616419913919, 0.00010036903534628722, -62.604838911140313, -56.183627034577746, -3.6273625977595579e-09, 0.87629252628631848, -47.902162685178212, -32.818923831929

In [59]:
import sciunit
modelp
import pandas as pd
attributes = [ i.attrs for i in some_tuples[0] ]
rheobases = [ i.rheobase for i in some_tuples[0] ]
#modelsp = [model.__class__(p) for p in parameters]
models = some_tuples[1]
sc = pd.DataFrame(scores[0])
for j,i in enumerate(models):
        i.name = attributes[j]
sc

,RheobaseTest,InputResistanceTest,TimeConstantTest,CapacitanceTest,RestingPotentialTest,InjectedCurrentAPWidthTest,InjectedCurrentAPAmplitudeTest,InjectedCurrentAPThresholdTest
vanilla,Ratio = 0.52,Z = -2.36,Z = -2.15,Z = -1.08,Z = 1.59,Z = 7.10,Z = -6.29,Z = 5.31


In [74]:
data = [ models[0].name ]
#data.append(str())
model_values0 = pd.DataFrame(data)        
model_values0
        
        #print(i.name)
#path = sciunit.ScoreArray(attributes)


AttributeError: 'str' object has no attribute 'keys'

In [76]:
rheobases[0]


67.5925925925926

In [77]:

sc1 = pd.DataFrame(scores[1])
sc1


,RheobaseTest,InputResistanceTest,TimeConstantTest,CapacitanceTest,RestingPotentialTest,InjectedCurrentAPWidthTest,InjectedCurrentAPAmplitudeTest,InjectedCurrentAPThresholdTest
vanilla,Ratio = 2.08,Z = 1.54,Z = -2.15,Z = -1.08,Z = 2.19,Z = 7.10,Z = -6.29,Insufficient Data


In [78]:
rheobases[1]


270.60185185185185

In [79]:
data=[ models[1].name ]
#data.append(str(rheobases[1]))
model_values1 = pd.DataFrame(data)        
model_values1

,C,a,b,c,d,k,v0,vpeak,vr,vt
0,9.93194568776e-05,0.671900527826,-4.47762006555e-09,-59.5125484902,0.0518254196574,0.00115780964351,-55.6996968732,39.3715553272,-67.3740489831,-36.813955781


In [80]:
models[1].name        

import pickle
import pandas as pd


try:
    ground_error = pickle.load(open('big_model_evaulated.pickle','rb'))
except:
    print('{0} it seems the error truth data does not yet exist, lets create it now '.format(str(False)))
    ground_error = list(futures.map(outils.func2map, ground_truth))
    pickle.dump(ground_error,open('big_model_evaulated.pickle','wb'))
#ground_error_nsga=list(zip(vmpop,pop,invalid_ind))
#pickle.dump(ground_error_nsga,open('nsga_evaulated.pickle','wb'))

#Get the differences between values obtained via brute force, and those obtained otherwise:
sum_errors = [ i[0] for i in ground_error ]
composite_errors = [ i[1] for i in ground_error ]
attrs = [ i[2] for i in ground_error ]
rheobase = [ i[3] for i in ground_error ]

indexs = [i for i,j in enumerate(sum_errors) if j==np.min(sum_errors) ][0]
indexc = [i for i,j in enumerate(composite_errors) if j==np.min(composite_errors) ][0]
#assert indexs == indexc
vmpop = some_tuples[0]
df_0 = pd.DataFrame([ (k,v,vmpop[0].attrs[k],float(v)-float(vmpop[0].attrs[k])) for k,v in ground_error[indexc][2].items() ])
df_1 = pd.DataFrame([ (k,v,vmpop[1].attrs[k],float(v)-float(vmpop[1].attrs[k])) for k,v in ground_error[indexc][2].items() ])



In [81]:


df_0

,0,1,2,3
0,vpeak,3.000000e+01,32.4416452854,-2.441645e+00
1,a,9.450000e-01,0.705821414254,2.391786e-01
2,vr,-5.000000e+01,-65.3603349633,1.536033e+01
3,b,-5.000000e-09,-4.75915425424e-09,-2.408457e-10


In [82]:
df_1


,0,1,2,3
0,vpeak,3.000000e+01,39.3715553272,-9.371555e+00
1,a,9.450000e-01,0.671900527826,2.730995e-01
2,vr,-5.000000e+01,-67.3740489831,1.737405e+01
3,b,-5.000000e-09,-4.47762006555e-09,-5.223799e-10
